In [52]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split


In [53]:
#load Dataset
df = pd.read_csv('dataset-1-KP.csv', encoding='UTF-8-SIG')

In [54]:
df.head()

,Order ID,Order Status,Order Substatus,Cancelation/Return Type,Normal or Pre-order,SKU ID,Seller SKU,Product Name,Variation,Quantity,...,Villages,Detail Address,Additional address information,Payment Method,Weight(kg),Product Category,Package ID,Seller Note,Checked Status,Checked Marked by
0,577066577177642591,Shipped,In Transit,NaN,Normal,1729668342918710874,NaN,T-Shirt Kaos Polos Pria Wanita Unisex Tilang I...,"Blue Denim, XL",1,...,De** **********,Pe*** ***** ********** **********,no ** ** *****,Bayar di tempat,0.020,T-shirts,1154259769080972895\t,NaN,Unchecked,NaN
1,577066535526761897,Shipped,Delivered,NaN,Normal,1729668342918710874,NaN,T-Shirt Kaos Polos Pria Wanita Unisex Tilang I...,"Blue Denim, XL",1,...,Ci**********,Me** ******* ***** ******* **** ***,NaN,Bayar di tempat,0.020,T-shirts,1154259518675387817\t,NaN,Unchecked,NaN
2,577066491222591233,Shipped,In Transit,NaN,Normal,1729649019553810010,200.0,T-Shirt Oversize Punk Unisex Tilang Indonesia ...,"HItam, M",1,...,Ko******,se***** ****** ** ******* **** *** ***** ** **...,ru*** *** ***** *** ** ** ** ****,Bayar di tempat,0.025,T-shirts,1154259374383269633\t,NaN,Unchecked,NaN
3,577055538496308305,Shipped,Delivered,NaN,Normal,1729669708476941914,NaN,T-Shirt Kaos Polos Pria Wanita Unisex Tilang I...,"Hitam, XL",1,...,Pe*********,ds********* **********,NaN,Virtual account transfer,0.020,T-shirts,1154216697779684433\t,NaN,Unchecked,NaN
4,577055274425812627,Shipped,Delivered,NaN,Normal,1729668310609331802,NaN,T-Shirt Baju Raja Jalanan Tilang Indonesia Vol. 1,L,1,...,Ci**** *****,Ko** ***** ***** ***** ***** **** **** **** * ...,NaN,Virtual account transfer,0.020,T-shirts,1154215556635461267\t,NaN,Unchecked,NaN


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 57 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Order ID                        149 non-null    int64  
 1   Order Status                    149 non-null    object 
 2   Order Substatus                 149 non-null    object 
 3   Cancelation/Return Type         14 non-null     object 
 4   Normal or Pre-order             149 non-null    object 
 5   SKU ID                          149 non-null    int64  
 6   Seller SKU                      92 non-null     float64
 7   Product Name                    149 non-null    object 
 8   Variation                       149 non-null    object 
 9   Quantity                        149 non-null    int64  
 10  Sku Quantity of return          149 non-null    int64  
 11  SKU Unit Original Price         149 non-null    object 
 12  SKU Subtotal Before Discount    149 

## Data Cleaning

In [56]:
#menghapus kolom yang tidak relevan
df_clean = df.drop(columns=['Order ID', 'Seller SKU', 'Tracking ID', 
                            'Cancelation/Return Type', 'Seller Note',
                            'Checked Marked by'], errors='ignore')

In [57]:
df_clean.head()

,Order Status,Order Substatus,Normal or Pre-order,SKU ID,Product Name,Variation,Quantity,Sku Quantity of return,SKU Unit Original Price,SKU Subtotal Before Discount,...,Regency and City,Districts,Villages,Detail Address,Additional address information,Payment Method,Weight(kg),Product Category,Package ID,Checked Status
0,Shipped,In Transit,Normal,1729668342918710874,T-Shirt Kaos Polos Pria Wanita Unisex Tilang I...,"Blue Denim, XL",1,0,IDR 120.000,IDR 120.000,...,Kota Tasikmalaya,Ma********,De** **********,Pe*** ***** ********** **********,no ** ** *****,Bayar di tempat,0.020,T-shirts,1154259769080972895\t,Unchecked
1,Shipped,Delivered,Normal,1729668342918710874,T-Shirt Kaos Polos Pria Wanita Unisex Tilang I...,"Blue Denim, XL",1,0,IDR 120.000,IDR 120.000,...,Kota Tasikmalaya,Bu********,Ci**********,Me** ******* ***** ******* **** ***,NaN,Bayar di tempat,0.020,T-shirts,1154259518675387817\t,Unchecked
2,Shipped,In Transit,Normal,1729649019553810010,T-Shirt Oversize Punk Unisex Tilang Indonesia ...,"HItam, M",1,0,IDR 135.000,IDR 135.000,...,Kota Tasikmalaya,Ci*******,Ko******,se***** ****** ** ******* **** *** ***** ** **...,ru*** *** ***** *** ** ** ** ****,Bayar di tempat,0.025,T-shirts,1154259374383269633\t,Unchecked
3,Shipped,Delivered,Normal,1729669708476941914,T-Shirt Kaos Polos Pria Wanita Unisex Tilang I...,"Hitam, XL",1,0,IDR 120.000,IDR 120.000,...,Tuban,Se*******,Pe*********,ds********* **********,NaN,Virtual account transfer,0.020,T-shirts,1154216697779684433\t,Unchecked
4,Shipped,Delivered,Normal,1729668310609331802,T-Shirt Baju Raja Jalanan Tilang Indonesia Vol. 1,L,1,0,IDR 147.000,IDR 147.000,...,Bandung,Ci******,Ci**** *****,Ko** ***** ***** ***** ***** **** **** **** * ...,NaN,Virtual account transfer,0.020,T-shirts,1154215556635461267\t,Unchecked


In [58]:
missing_values = df_clean.isnull().sum()
print(missing_values)

Order Status                        0
Order Substatus                     0
Normal or Pre-order                 0
SKU ID                              0
Product Name                        0
Variation                           0
Quantity                            0
Sku Quantity of return              0
SKU Unit Original Price             0
SKU Subtotal Before Discount        0
SKU Platform Discount               0
SKU Seller Discount                 0
SKU Subtotal After Discount         0
Shipping Fee After Discount         0
Original Shipping Fee               0
Shipping Fee Seller Discount        0
Shipping Fee Platform Discount      0
Payment platform discount         147
Buyer Service Fee                   0
Taxes                               0
Order Amount                        0
Order Refund Amount               135
Created Time                        0
Paid Time                           0
RTS Time                            0
Shipped Time                        0
Delivered Ti

In [59]:
missing_percentage = (df_clean.isnull().sum()/len(df_clean))*100
missing_percentage

Order Status                       0.000000
Order Substatus                    0.000000
Normal or Pre-order                0.000000
SKU ID                             0.000000
Product Name                       0.000000
Variation                          0.000000
Quantity                           0.000000
Sku Quantity of return             0.000000
SKU Unit Original Price            0.000000
SKU Subtotal Before Discount       0.000000
SKU Platform Discount              0.000000
SKU Seller Discount                0.000000
SKU Subtotal After Discount        0.000000
Shipping Fee After Discount        0.000000
Original Shipping Fee              0.000000
Shipping Fee Seller Discount       0.000000
Shipping Fee Platform Discount     0.000000
Payment platform discount         98.657718
Buyer Service Fee                  0.000000
Taxes                              0.000000
Order Amount                       0.000000
Order Refund Amount               90.604027
Created Time                    

In [60]:
df_cleaned = df_clean.dropna(axis=1)

In [61]:
missing_values = df_cleaned.isnull().sum()
print(missing_values)

Order Status                      0
Order Substatus                   0
Normal or Pre-order               0
SKU ID                            0
Product Name                      0
Variation                         0
Quantity                          0
Sku Quantity of return            0
SKU Unit Original Price           0
SKU Subtotal Before Discount      0
SKU Platform Discount             0
SKU Seller Discount               0
SKU Subtotal After Discount       0
Shipping Fee After Discount       0
Original Shipping Fee             0
Shipping Fee Seller Discount      0
Shipping Fee Platform Discount    0
Buyer Service Fee                 0
Taxes                             0
Order Amount                      0
Created Time                      0
Paid Time                         0
RTS Time                          0
Shipped Time                      0
Delivered Time                    0
Cancelled Time                    0
Fulfillment Type                  0
Warehouse Name              

In [62]:
df_cleaned.head()

,Order Status,Order Substatus,Normal or Pre-order,SKU ID,Product Name,Variation,Quantity,Sku Quantity of return,SKU Unit Original Price,SKU Subtotal Before Discount,...,Country,Province,Regency and City,Districts,Villages,Detail Address,Payment Method,Weight(kg),Product Category,Package ID
0,Shipped,In Transit,Normal,1729668342918710874,T-Shirt Kaos Polos Pria Wanita Unisex Tilang I...,"Blue Denim, XL",1,0,IDR 120.000,IDR 120.000,...,Indonesia,Jawa Barat,Kota Tasikmalaya,Ma********,De** **********,Pe*** ***** ********** **********,Bayar di tempat,0.020,T-shirts,1154259769080972895\t
1,Shipped,Delivered,Normal,1729668342918710874,T-Shirt Kaos Polos Pria Wanita Unisex Tilang I...,"Blue Denim, XL",1,0,IDR 120.000,IDR 120.000,...,Indonesia,Jawa Barat,Kota Tasikmalaya,Bu********,Ci**********,Me** ******* ***** ******* **** ***,Bayar di tempat,0.020,T-shirts,1154259518675387817\t
2,Shipped,In Transit,Normal,1729649019553810010,T-Shirt Oversize Punk Unisex Tilang Indonesia ...,"HItam, M",1,0,IDR 135.000,IDR 135.000,...,Indonesia,Jawa Barat,Kota Tasikmalaya,Ci*******,Ko******,se***** ****** ** ******* **** *** ***** ** **...,Bayar di tempat,0.025,T-shirts,1154259374383269633\t
3,Shipped,Delivered,Normal,1729669708476941914,T-Shirt Kaos Polos Pria Wanita Unisex Tilang I...,"Hitam, XL",1,0,IDR 120.000,IDR 120.000,...,Indonesia,Jawa Timur,Tuban,Se*******,Pe*********,ds********* **********,Virtual account transfer,0.020,T-shirts,1154216697779684433\t
4,Shipped,Delivered,Normal,1729668310609331802,T-Shirt Baju Raja Jalanan Tilang Indonesia Vol. 1,L,1,0,IDR 147.000,IDR 147.000,...,Indonesia,West Java,Bandung,Ci******,Ci**** *****,Ko** ***** ***** ***** ***** **** **** **** * ...,Virtual account transfer,0.020,T-shirts,1154215556635461267\t


### Perbedaaan sebelum dan setelah data cleaning

In [66]:
missing_values_before = df.isnull().sum().sum()
print('Jumlah Missing Value sebelum dibersihkan: ')
print(missing_values_before)

Jumlah Missing Value sebelum dibersihkan: 
1263


In [68]:
missing_values_after = df_cleaned.isnull().sum().sum()
print('Jumlah Missing Value setelah dibersihkan: ')
print(missing_values_after)

Jumlah Missing Value setelah dibersihkan: 
0


## Analisis Deskriptif

In [75]:
# Menghitung total penjualan per kategori produk
sales_per_category = df_cleaned.groupby('Product Category')['SKU Subtotal After Discount'].sum().sort_values(ascending=False)
print(sales_per_category)

Product Category
Tops                              IDR 98.999IDR 94.049IDR 94.587
T-shirts       IDR 82.999IDR 82.799IDR 99.650IDR 82.999IDR 91...
Polo Shirts    IDR 118.000IDR 123.119IDR 246.236IDR 118.750ID...
Name: SKU Subtotal After Discount, dtype: object


In [74]:
# Membersihkan kolom 'SKU Subtotal After Discount' dengan menghapus huruf 'IDR' dan mengonversinya ke tipe numerik
df_clean['SKU Subtotal After Discount'] = df_clean['SKU Subtotal After Discount'].str.replace('IDR ', '').astype(float)

# Menghitung total penjualan per kategori produk
sales_per_category = df.groupby('Product Category')['SKU Subtotal After Discount'].sum().sort_values(ascending=False)
print(sales_per_category)

AttributeError: Can only use .str accessor with string values!